### View Rendered Notebook [Here](https://laughing-goldstine-368323.netlify.app/gallery/demo) 
### _(Note rendering is slow due to the size of the dataset. For the same notebook with a smaller subset of the data see [mini-demo](https://laughing-goldstine-368323.netlify.app/gallery/demo-mini))_
# Demo of PDL's Companies Dataset

This is a quick notebook to demonstrate some of the things you can do with the PDL's companies dataset. We'll use a subset of the dataset containing just the companies based within the US (to limit resource usage and keep this notebook somewhat interactive). Below is an overview of the things we'll cover in this notebook:

## Overview
0. Brief Exploratory Analysis 
1. Show a map of the top 10 largest companies in the US
2. Show the relationship between company age and company size
3. Show the most common industry in pittsburgh

# Getting setup

First let's get our environment setup and load in the necessary modules and data

In [ ]:
# Imports
import os
import json
import pandas as pd
import numpy as np 
import folium
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px

# Dataset path (subset of full dataset containing first 1.8 million US-based companies)
# We'll take just a subset of this dataset in the next cell
data_dir = "data"
dataset_file = "2mm_companies_us.csv"
dataset_path = os.path.join( data_dir, dataset_file )

In [ ]:
# Read dataset (note: this is could take ~30 sec to load)
df = pd.read_csv(dataset_path, error_bad_lines=False)
df

# 0. Exploratory Analysis

Next, let's take a quick look at the dataset to better understand the data it contains

#### Number of Companies in Each Industry

In [ ]:
fig = px.histogram(df, x='industry', title="Number of Companies in Each Industry")

fig.update_xaxes(title_text = "")
fig.update_yaxes(title_text = "Count")

fig.show()

Seems like the most common industries represented in this dataset are:
1. Construction
2. Information Technology and Services
3. Marketing and Advertising


#### Number of Companies Founded Each Year

In [ ]:
fig = px.histogram(df, x='year founded', range_x=(1800,2020), nbins=220, 
                   title='Number of Companies Founded Each Year')

fig.update_xaxes(title_text = "Year Founded")
fig.update_yaxes(title_text = "Count")

fig.show()

It's interesting to note that the majority of companies in the dataset were founded between 2010-2015; after that there is noticeably less data for more recent companies.

#### Number of Companies in Each US City

In [ ]:
us_df = df[ df['locality'].notnull() ] # remove missing localities
us_df = us_df[ us_df['locality'].str.contains('united states') ] 
us_counts = us_df['locality'].value_counts() 

fig = plotly.subplots.make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=np.arange(us_counts.shape[0]), y=us_counts, 
               hovertext=us_counts.index, mode='markers', name="Count"), 
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(us_counts.shape[0]), y=np.cumsum(us_counts)/us_counts.sum(),
               hovertext=us_counts.index, mode='markers', name='Cumulative Density'), 
    row=1, col=2
)

fig.update_layout(title_text="Number of Companies in Each US City")
fig.update_layout(hovermode='closest')
fig.update_xaxes(title_text = "Locality Index")
fig.update_yaxes(title_text = "Count", row=1, col=1)
fig.update_yaxes(title_text = "Cumulative Densitry", row=1, col=2)
fig.show()

print("Total Number of US Companies: {}".format(us_counts.sum()))

This figure shows the Power Law Distribution of companies within US cities (where most of the companies are concentrated in relatively few cities). 

While we're at it, let's also see what this looks like geographically.

In [ ]:
# Read latlon lookup table
locality_to_latlon = {}
lookup_table_file = 'locality_to_latlon.json'
lookup_table_path = os.path.join( data_dir, lookup_table_file )
with open(lookup_table_path, 'r', encoding='utf-8') as f:
    locality_to_latlon = json.load(f)

# The ranges for number of companies in a city 
# (each city will fall into one of these ranges)
sizes = [(0, 500), (500, 1000), (1000, 5000), (5000, 10000), (10000, 100000)]
scale = 100

# Create a dataframe with all the relevant info for plotting
df_plot = pd.DataFrame(columns=['locality', 'lat', 'lon', 'size', 'sizeClass'])

df_plot['locality'] = us_counts.index
df_plot['size'] = us_counts.values
df_plot['sizeClass'] = np.digitize(df_plot['size'], bins=[size[0] for size in sizes])

def lookupLatLon(row):
  locality = row['locality']
  if locality in locality_to_latlon:
    lat = locality_to_latlon[row['locality']][0]
    lon = locality_to_latlon[row['locality']][1]
    return (lat, lon)
  else:
    return (0, 0)

df_plot['lat'] = df_plot.apply( lambda row: lookupLatLon(row)[0], axis=1 )
df_plot['lon'] = df_plot.apply( lambda row: lookupLatLon(row)[1], axis=1 )


# Plot our map
fig = go.Figure()

colors = ["royalblue","lightseagreen","orange","purple","crimson"]

for ii in range(len(sizes))[::-1]: # Plot largest sizes first
    df_sub = df_plot[ df_plot['sizeClass'] == ii+1]#.iloc[:10000]

    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub[['locality', 'size']],
        marker = dict(
            size = df_sub['size']/scale,
            color = colors[ii],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(sizes[ii][0],sizes[ii][1]))
    )

fig.update_layout(
        title_text = 'Number of Companies in US Cities',
        showlegend = True,
        width=1000, height=1000,
        geo = dict(
            scope = 'usa',
        )
    )

fig.show()

# 1. Map the 10 Largest Companies in the US

Now as our first real task, let's find the 10 largest companies in the US and plot them on a map.

In [ ]:
# Map the 10 Largest Companies in the US

# Sort companies by size
us_companies = df[ df['country'] == 'united states']
company_sizes = us_companies['total employee estimate'].values
sorted_idxs = np.argsort(company_sizes)[::-1] # descending order
sorted_idxs = sorted_idxs[:10] # top 10

# Plot these companies on a map
m = folium.Map(location=[39.8282, -98.5795], zoom_start=4)

count = 1
print("The 10 Largest US Companies are:")

for idx, company in us_companies.iloc [ sorted_idxs ].iterrows():
    name = company['name']
    locality = company['locality']
    size = int ( company['total employee estimate'] )
    if locality in locality_to_latlon:
        latlon = locality_to_latlon[locality]
        latlon = [float(x) for x in latlon]
        folium.Circle(location=latlon,
                      popup="{}. {} - {} [{} employees]".format(count, name, locality, size),
                      radius=size/5,
                      fill=True,
                   ).add_to(m)
    else:
        print("Could not find latlon for {}".format(name))

    print("  {}. {} - {} [{} employees]".format(count, name, locality, size))
    count += 1

# Show map:
m

# 2. Relationship between Company Age and Company Size

Next, let's take a look and see if we can find any relationship between the age of a company and it's size.

In [ ]:
# Plot company size as a function of it's age:

# A little data cleaning to get remove missing/null values
clean_df = df[ df['year founded'].notnull() ]
clean_df = clean_df[ clean_df['total employee estimate'].notnull() ]

# Plot
fig = px.scatter(clean_df, x='year founded', y='total employee estimate', 
                 hover_name='name', 
                 title="Company Age vs Size",
                 labels={"year founded": "Year Founded", "total employee estimate": "Total Employee Estimate"})

fig.update_xaxes(range=[1800, 2020])

fig.show()

Perhaps unsurprisingly, there doesn't seem to be any correlation between a company's size and the year in which it was founded. In fact, what seems to stand out the most are the outliers, where unexpectedly large companies seem to pop up with equal probability across the years. 

# 3. Comparing Industries Between Cities

Now, let's look at the how industries vary across different cities. We know from our [previous cell](https://colab.research.google.com/drive/1K-08RG_jgfyDvLrkHzcfOHwwCLANE6hz#scrollTo=wx4fv-EN4wNj&line=1&uniqifier=1) that Chicago, San Francisco, and Houston have a similar number of companies, so let's see how they are distributed across industries.

In [ ]:
sf_companies = df [ df['locality'] == 'san francisco, california, united states']
chi_companies = df[ df['locality'] == 'chicago, illinois, united states' ]
hou_companies = df[ df['locality'] == 'houston, texas, united states' ]

fig = go.Figure()

go.Histogram( )

fig.add_trace(go.Histogram(x=hou_companies['industry'], name='HOU'))
fig.add_trace(go.Histogram(x=sf_companies['industry'], name='SF'))
fig.add_trace(go.Histogram(x=chi_companies['industry'], name='CHI'))


# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()


From this figure, it seems like Houston stands out for it's oil & energy as well as it's construction industries, while Chicago seems to have more companies in it's marketing & advertising as well as it's financial services industries. Unsurprisingly, San Francisco dominates in its internet, computer software, and information technology industries.

# Wrapping Up

As you can see, there are a lot of interesting findings and useful insights that can be found by looking into this dataset. Hopefully, this gives you some ideas!